In [475]:
%matplotlib inline
from numpy import *
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from numpy import random
from numba import jit,njit,prange
from scipy import fftpack 
from Plot import Snap_Save,TwilightPlot,BinaryPlot,RdBuPlot,cbar 
from PlotFuncs import MyDoublePlot,MySquarePlot,cbar 
from Laplacian import Laplacian_2D, Gradient_2D
from StringID import cores_pi2
from scipy.fft import fft, ifft, fft2,fftshift
import time
#import os
#os.environ['NUMBA_DISABLE_INTEL_SVML']  = '1'
#import numba
mov_dir = 'mov/2D/' # Directory for the snapshots 

#Physical parameters
fa = 1 # In natural fa units
lambdaPRS = 1 # Quartic coupling 
ms = math.sqrt(lambdaPRS)*fa # Saxion mass for PRS in ADM units
Mpl = 2.4*10**6.0# Normalized to fa = 10^11 GeV
# Code parameters 
N = 256 # Number of grid points
L = 256 # Comoving 
DeltaRatio = 1.0/3.0
Delta = L/N 
Delta_tau = DeltaRatio*Delta
# Dimensionless spacing and time spacing 
dx = ms*Delta 
dtau = ms*Delta_tau
t_evol = dtau/DeltaRatio - dtau # Conformal time initial and in the running, to set tau = 1 in the first loop
t = t_evol**2.0/2.0 
light_time = int(0.5*N*dx/dtau)
era = 1 # 2 for matter domination, 1 for radiation domination (in PRS trick)

# TEMPERATURE STUFF
gstar = 106.75
T0 = math.sqrt(Mpl*90.0/(gstar*pi*pi))
#rho0 = 3*Mpl
#prop_const = (30*rho0/(gstar*pi*pi))**1.0/4.0# between T and inverse scale factor (or conf time)

# MAYBE IMPLEMENT PRINTING ON EXTERNAL FILE THE PARAMETERS 
#param_file = open("Output/Sim_parameters.txt","w+")

print(f'Saxion mass: {ms} fa')
print(f'Comoving core size: {ms**(-1.0)} fa\n')
print('Dimensionless Comoving space step:', dx)
print('Dimensionless Comoving time step:', dtau)
print('Dimensionless cosmic time step:', dtau**2.0/2.0,'\n')
print('Dimensionful Comoving space step:', Delta)
print('Dimensionful Comoving time step:', Delta_tau)
print(f'Dimensionful cosmic time step: {(dtau**2.0/2.0)/ms}\n')
print('Initial conformal time:', dtau/DeltaRatio)
print('Initial cosmic time:', (dtau/DeltaRatio)**2.0/2.0)
print('Initial scale factor:',dtau/DeltaRatio)
print(f'Initial log: {log((t_evol+dtau)*ms)}')
print(f'Initial temperature (fa): {T0}\n')
print('Number of time steps in the code:', light_time,'\n')
print('Final conformal time:', light_time*dtau/ms)
print('Final cosmic time:', (light_time*dtau/ms)**2.0/2.0)
print('Final scale factor:' , light_time*dtau/ms)
print('Final log:', log(ms*(light_time*dtau)),'\n')
print('Initial T:', T0)
print('Final T:', T0/(ms*light_time*dtau)**2.0)
print('kmax:',pi*N/L)
print('kmin:',2*pi/L)

Saxion mass: 1.0 fa
Comoving core size: 1.0 fa

Dimensionless Comoving space step: 1.0
Dimensionless Comoving time step: 0.3333333333333333
Dimensionless cosmic time step: 0.05555555555555555 

Dimensionful Comoving space step: 1.0
Dimensionful Comoving time step: 0.3333333333333333
Dimensionful cosmic time step: 0.05555555555555555

Initial conformal time: 1.0
Initial cosmic time: 0.5
Initial scale factor: 1.0
Initial log: 0.0
Initial temperature (fa): 452.78607391161006

Number of time steps in the code: 384 

Final conformal time: 128.0
Final cosmic time: 8192.0
Final scale factor: 128.0
Final log: 4.852030263919617 

Initial T: 452.78607391161006
Final T: 0.02763586876901917
kmax: 3.141592653589793
kmin: 0.02454369260617026


In [476]:
#start = time.time()
th = 2*pi*random.uniform(-1,1,size=(N,N)) # In Fourier space
init = ifft(th)
phi1 = init.real
phi2 =init.imag
phidot1 = zeros(shape=(N,N))
phidot2 = zeros(shape=(N,N))

# IMPLEMENT THERMAL INITIAL POWER SPECTRUM  
#
#wnoise = 2*pi*random.uniform(size=(N,N)) # Fourier space realization of white noise
#dk = 2*pi/N
#kmax = pi/dx
#kmin = 2*pi/L
#kx = linspace(-pi+dk,pi,N)
#ky = kx
#meffsquared=lambdaPRS*T0**2.0/3.0
#KX,KY = meshgrid(kx,ky)
#K = sqrt(KX**2+KY**2)
#
#def thermal_ps(L,K,meffsquared,T0):
#    omegak = sqrt(K**2.0 + meffsquared)
#    bose = 1/(exp(omegak/T0)-1)
#    spectrum = sqrt(L*bose/omegak)
#    return spectrum
#
#spectrum1 = thermal_ps(L,K,meffsquared,T0)
#init = ifft(spectrum1*whitenoise)
#phi1 = init.real
#phi2 = init.imag

# KERNEL
K1 = Laplacian_2D(phi1,dx,N) - 2*era/t_evol*phidot1 - lambdaPRS*phi1*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))
K2 = Laplacian_2D(phi2,dx,N) - 2*era/t_evol*phidot2 - lambdaPRS*phi2*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))

#Hpatches = open("Files/Times.txt","w+")
#Hpatches.write('Simulation time, Conformal time, Cosmic time, Hubble patches, Log, Temperature \n')
for tstep in range(0,light_time-int(1/DeltaRatio)+1):
    # DRIFT
    phi1 = phi1 + dtau*(phidot1 + 0.5*K1*dtau)
    phi2 = phi2 + dtau*(phidot2 + 0.5*K2*dtau)
    # UPDATE
    t_evol = t_evol + dtau
    K1_next = Laplacian_2D(phi1,dx,N) - 2*era/t_evol*phidot1 - lambdaPRS*phi1*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))  
    K2_next = Laplacian_2D(phi2,dx,N) - 2*era/t_evol*phidot2 - lambdaPRS*phi2*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))  
    # The last term gives overflow warning 
    #KICK
    phidot1 = phidot1 + 0.5*(K1 + K1_next)*dtau
    phidot2 = phidot2 + 0.5*(K2 + K2_next)*dtau
    # UPDATE
    K1 = 1.0*K1_next
    K2 = 1.0*K2_next
    axion_field=arctan2(phi1,phi2)
    saxion_field = sqrt(phi1**2.0+phi2**2.0)
    # TIMES 
    #print('%d %f %f %f %f %f \n' % (tstep,t_evol,t_evol**2.0/2.0,L/t_evol,log(ms*t_evol),T0*prop_const/(prop_const*t_evol**2.0)))
    #Hpatches.write('%d %f %f %f %f %f \n' % (tstep,t_evol,t_evol**2.0/2.0,L/t_evol,log(ms*t_evol),T0/(t_evol**2.0)))
#Hpatches.close()
#fig,ax = TwilightPlot(axion_field,t_evol)
#cores_pi2(axion_field,N,1)
#Snap_Save(fig,tstep,mov_dir)
#print('It took', time.time()-start, 'seconds.')

<ipython-input-476-6c22dfd9f794>:42: RuntimeWarning: overflow encountered in multiply
  K1_next = Laplacian_2D(phi1,dx,N) - 2*era/t_evol*phidot1 - lambdaPRS*phi1*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))
<ipython-input-476-6c22dfd9f794>:43: RuntimeWarning: overflow encountered in multiply
  K2_next = Laplacian_2D(phi2,dx,N) - 2*era/t_evol*phidot2 - lambdaPRS*phi2*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))
<ipython-input-476-6c22dfd9f794>:42: RuntimeWarning: overflow encountered in square
  K1_next = Laplacian_2D(phi1,dx,N) - 2*era/t_evol*phidot1 - lambdaPRS*phi1*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))
<ipython-input-476-6c22dfd9f794>:42: RuntimeWarning: invalid value encountered in subtract
  K1_next = Laplacian_2D(phi1,dx,N) - 2*era/t_evol*phidot1 - lambdaPRS*phi1*(phi1**2.0+phi2**2.0 - 1 + T0**2.0/(3.0*t_evol**4.0))
<ipython-input-476-6c22dfd9f794>:43: RuntimeWarning: overflow encountered in square
  K2_next = Laplacian_2D(phi2,dx,N) - 2*era/t_evo